In [32]:
# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Predicting Movie Review Sentiment with BERT on TF Hub

If you’ve been following Natural Language Processing over the past year, you’ve probably heard of BERT: Bidirectional Encoder Representations from Transformers. It’s a neural network architecture designed by Google researchers that’s totally transformed what’s state-of-the-art for NLP tasks, like text classification, translation, summarization, and question answering.

Now that BERT's been added to [TF Hub](https://www.tensorflow.org/hub) as a loadable module, it's easy(ish) to add into existing Tensorflow text pipelines. In an existing pipeline, BERT can replace text embedding layers like ELMO and GloVE. Alternatively, [finetuning](http://wiki.fast.ai/index.php/Fine_tuning) BERT can provide both an accuracy boost and faster training time in many cases.

Here, we'll train a model to predict whether an IMDB movie review is positive or negative using BERT in Tensorflow with tf hub. Some code was adapted from [this colab notebook](https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb). Let's get started!

In [33]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In addition to the standard libraries we imported above, we'll need to install BERT's python package.

In [34]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

Below, we'll set an output directory location to store our model output and checkpoints. This can be a local directory, in which case you'd set OUTPUT_DIR to the name of the directory you'd like to create. If you're running this code in Google's hosted Colab, the directory won't persist after the Colab session ends.

Alternatively, if you're a GCP user, you can store output in a GCP bucket. To do that, set a directory name in OUTPUT_DIR and the name of the GCP bucket in the BUCKET field.

Set DO_DELETE to rewrite the OUTPUT_DIR if it exists. Otherwise, Tensorflow will load existing model checkpoints from that directory (if they exist).

In [35]:
!mkdir bert_category_confusion

mkdir: cannot create directory ‘bert_category_confusion’: File exists


In [36]:
OUTPUT_DIR = "bert_category_confusion"

In [37]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'bert_category_confusion'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: bert_category_confusion *****


# Data

First, let's download the dataset, hosted by Stanford. The code below, which downloads, extracts, and imports the IMDB Large Movie Review Dataset, is borrowed from [this Tensorflow tutorial](https://www.tensorflow.org/hub/tutorials/text_classification_with_tf_hub).

In [38]:
from tensorflow import keras
import os
import re

In [39]:
Conventional = ["Generic correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Choice of tense", "Tense form", "Voice", "Modals", "Verb pattern", "Intransitive verb", "Transitive verb", "Reflexive verb", "Verb with as", "Ambitransitive verb", "Two verbal forms in the predicate", "Verb + Infinitive", "Verb + Gerund", "Verb + Infinitive OR Gerund", "Verb + Bare Infinitive", "Verb + Object/Addressee + Bare Infinitive", "Infinitive Restoration Alternation", "Verb + Participle", "Get + participle", "Complex-object verb", "Verbal idiom", "Prepositional or phrasal verb", "Dative verb with alternation", "Verb followed by a clause", "Verb + that/WH + Clause", "Verb + if/whether + clause", "Verb + that + Subjunctive clause", "Verb + it + Conj + Clause", "Participial construction", "Infinitive construction", "Gerund phrase", "Nouns", "Countable/uncountable", "Prepositional noun", "Possessive form of a noun", "Noun as an attribute", "Noun + Infinitive", "Noun number", "Prepositions", "Conjunctions", "Adjectives", "Comparative degree of adjectives", "Superlative degree of adjectives", "Prepositional adjective", "Adjective as a collective noun", "Adverbs", "Comparative degree of adverbs", "Superlative degree of adverbs", "Prepositional adverb", "Numerals", "Pronouns", "Agreement", "Word order", "Standard word order", "Emphatic shift", "Cleft sentence", "Interrogative word order", "Incomplete sentence", "Exclamation", "Title structure", "Note structure", "Conditionals", "Attributes", "Relative clause", "Defining relative clause", "Non-defining relative clause", "Coordinate relative clause", "Attributive participial construction", "Parallel constructions", "Negation", "Comparative construction", "Numerical comparison", "Confusion of structures", "Vocabulary", "Word choice", "Choice of lexical item", "Words often confused", "Choice of a part of lexical item", "Absence of certain components of a collocation", "Redundant word(s)", "Word formation", "Derivational affixes", "Formational suffix", "Formational prefix", "Confusion of categories", "Compound word", "Discourse", "Referential device", "Coherence", "Linking device", "Inappropriate register", "Absence of a component in clause or sentence", "Redundant component in clause or sentence", "Absence of necessary explanation or detail", "Deletion"]
Tags = ["Correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Tense_choice", "Tense_form", "Voice", "Modals", "Verb_pattern", "Intransitive", "Transitive", "Reflexive_verb", "Presentation", "Ambitransitive", "Two_in_a_row", "Verb_Inf", "Verb_Gerund", "Verb_Inf_Gerund", "Verb_Bare_Inf", "Verb_object_bare", "Restoration_alter", "Verb_part", "Get_part", "Complex_obj", "Verbal_idiom", "Prepositional_verb", "Dative", "Followed_by_a_clause", "that_clause", "if_whether_clause", "that_subj_clause", "it_conj_clause", "Participial_constr", "Infinitive_constr", "Gerund_phrase", "Nouns", "Countable_uncountable", "Prepositional_noun", "Possessive", "Noun_attribute", "Noun_inf", "Noun_number", "Prepositions", "Conjunctions", "Adjectives", "Comparative_adj", "Superlative_adj", "Prepositional_adjective", "Adj_as_collective", "Adverbs", "Comparative_adv", "Superlative_adv", "Prepositional_adv", "Numerals", "Pronouns", "Agreement_errors", "Word_order", "Standard", "Emphatic", "Cleft", "Interrogative", "Abs_comp_clause", "Exclamation", "Title_structure", "Note_structure", "Conditionals", "Attributes", "Relative_clause", "Defining", "Non_defining", "Coordinate", "Attr_participial", "Lack_par_constr", "Negation", "Comparative_constr", "Numerical", "Confusion_of_structures", "Vocabulary", "Word_choice", "lex_item_choice", "Often_confused", "lex_part_choice", "Absence_comp_colloc", "Redundant", "Derivation", "Formational_affixes", "Suffix", "Prefix", "Category_confusion", "Compound_word", "Discourse", "Ref_device", "Coherence", "Linking_device", "Inappropriate_register", "Absence_comp_sent", "Redundant_comp", "Absence_explanation", "delete"]
translate_dict = {e[0]: e[1] for e in zip(Conventional, Tags)}

error_type = "Confusion of categories" #@param ["Spelling", "Choice of lexical item", "Deletion", "Prepositions", "Agreement", "Noun number", "Confusion of categories", "Referential device", "Capitalisation", "Words often confused"]
error_ratio = "ToFifteen" #@param ["ToFifteen", "AugmentedRatio"]

error_type = translate_dict[error_type]

It's about time we named our model

In [40]:
modelname = "Category_confusion_ToFifteen" # @param {type:"string"}

In [41]:
import shutil

filename = error_ratio+"/train/"+error_type+".json"
shutil.copy2('./Datasets/'+filename,'.')
trn = pd.read_json(error_type+".json").reset_index(drop=True)
trn["is_error"] = trn["is_error"].astype(int)

filename = error_ratio+"/test/"+error_type+".json"
shutil.copy2('./Datasets/'+filename,'./'+error_type+'_test.json')
tst = pd.read_json(error_type+"_test.json").reset_index(drop=True)
tst["is_error"] = tst["is_error"].astype(int)

In [42]:
len(trn)

25715

To keep training fast, we'll take a sample of 5000 train and test examples, respectively.

In [43]:
train = trn
test = tst

In [44]:
train.columns

Index(['context', 'is_error', 'path', 'substring'], dtype='object')

For us, our input data is the 'sentence' column and our label is the 'polarity' column (0, 1 for negative and positive, respecitvely)

In [45]:
DATA_COLUMN = 'context'
SUBSTR_COLUMN = 'substring'
LABEL_COLUMN = 'is_error'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

#Data Preprocessing
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [46]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[SUBSTR_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[SUBSTR_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [47]:
train_InputExamples

0        <bert.run_classifier.InputExample object at 0x...
1        <bert.run_classifier.InputExample object at 0x...
2        <bert.run_classifier.InputExample object at 0x...
3        <bert.run_classifier.InputExample object at 0x...
4        <bert.run_classifier.InputExample object at 0x...
5        <bert.run_classifier.InputExample object at 0x...
6        <bert.run_classifier.InputExample object at 0x...
7        <bert.run_classifier.InputExample object at 0x...
8        <bert.run_classifier.InputExample object at 0x...
9        <bert.run_classifier.InputExample object at 0x...
10       <bert.run_classifier.InputExample object at 0x...
11       <bert.run_classifier.InputExample object at 0x...
12       <bert.run_classifier.InputExample object at 0x...
13       <bert.run_classifier.InputExample object at 0x...
14       <bert.run_classifier.InputExample object at 0x...
15       <bert.run_classifier.InputExample object at 0x...
16       <bert.run_classifier.InputExample object at 0x.

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):


1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

Happily, we don't have to worry about most of these details.




To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [48]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1"

class NonMaskOmittingTokenizer(bert.tokenization.FullTokenizer):
  def tokenize(self, text):
    split_tokens = []
    for token in self.basic_tokenizer.tokenize(text):
      for sub_token in self.wordpiece_tokenizer.tokenize(token):
        split_tokens.append(sub_token)

    for index, item in enumerate(split_tokens):
      if index >= len(split_tokens)-2:
        break
      if item == '[' and split_tokens[index + 1] == 'MA' and split_tokens[index + 2] == '##S' and split_tokens[index + 3] == '##K' and split_tokens[index + 4] == ']':
        split_tokens[index] = "[MASK]"
        del split_tokens[index + 1]
        del split_tokens[index + 1]
        del split_tokens[index + 1]
        del split_tokens[index + 1]

    return split_tokens

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return NonMaskOmittingTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0509 12:12:44.878618 140481974605568 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [49]:
tokenizer.tokenize("This here's an example of using the BERT [MASK] tokenizer")

['This',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'B',
 '##ER',
 '##T',
 '[MASK]',
 'token',
 '##izer']

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [50]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 25715


I0509 12:12:45.421538 140481974605568 tf_logging.py:115] Writing example 0 of 25715


INFO:tensorflow:*** Example ***


I0509 12:12:45.425445 140481974605568 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: None


...

I0509 12:13:11.542894 140481974605568 tf_logging.py:115] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1


I0509 12:13:11.544111 140481974605568 tf_logging.py:115] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1


INFO:tensorflow:label: 0 (id = 0)


I0509 12:13:11.545387 140481974605568 tf_logging.py:115] label: 0 (id = 0)


#Creating a model

Now that we've prepared our data, let's focus on building a model. `create_model` does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called [fine-tuning](http://wiki.fast.ai/index.php/Fine_tuning).

In [51]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [52]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [66]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 12.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [67]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS) * 2
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

print(num_train_steps)

19286


In [68]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [69]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_master': '', '_eval_distribute': None, '_save_checkpoints_steps': 500, '_tf_random_seed': None, '_service': None, '_evaluation_master': '', '_save_summary_steps': 100, '_task_type': 'worker', '_num_worker_replicas': 1, '_device_fn': None, '_model_dir': 'bert_category_confusion', '_is_chief': True, '_log_step_count_steps': 100, '_protocol': None, '_task_id': 0, '_train_distribute': None, '_keep_checkpoint_max': 5, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_num_ps_replicas': 0, '_save_checkpoints_secs': None, '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbf1ed11e10>, '_experimental_distribute': None}


I0509 13:26:24.240514 140481974605568 tf_logging.py:115] Using config: {'_master': '', '_eval_distribute': None, '_save_checkpoints_steps': 500, '_tf_random_seed': None, '_service': None, '_evaluation_master': '', '_save_summary_steps': 100, '_task_type': 'worker', '_num_worker_replicas': 1, '_device_fn': None, '_model_dir': 'bert_category_confusion', '_is_chief': True, '_log_step_count_steps': 100, '_protocol': None, '_task_id': 0, '_train_distribute': None, '_keep_checkpoint_max': 5, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_num_ps_replicas': 0, '_save_checkpoints_secs': None, '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbf1ed11e10>, '_experimental_distribute': None}


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [70]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [71]:
print('Beginning Training!')
current_time = datetime.now()

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3)

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    with tf.device('/gpu:0'):
        estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


I0509 13:26:36.844294 140481974605568 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0509 13:26:41.302967 140481974605568 tf_logging.py:115] Saver not created because there are no variables in the graph to restore
/home/ivantorubarov/anaconda3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving checkpoints for 19286 into bert_category_confusion/model.ckpt.


I0509 14:59:03.389896 140481974605568 tf_logging.py:115] Saving checkpoints for 19286 into bert_category_confusion/model.ckpt.


INFO:tensorflow:Loss for final step: 4.731085e-06.


I0509 14:59:06.647620 140481974605568 tf_logging.py:115] Loss for final step: 4.731085e-06.


Training took time  1:32:40.959392


Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

Now let's use our test data to see how well our model did:

In [72]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [79]:
estimator.evaluate(input_fn=test_input_fn, steps=None)
# estimator.evaluate(input_fn=test_input_fn, checkpoint_path="./bert_category_confusion/model.ckpt-17320", steps=None)

INFO:tensorflow:Calling model_fn.


I0509 16:26:41.005677 140481974605568 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0509 16:26:45.580427 140481974605568 tf_logging.py:115] Saver not created because there are no variables in the graph to restore
/home/ivantorubarov/anaconda3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 19286: auc = 0.64966416, eval_accuracy = 0.94167054, f1_score = 0.40381554, false_negatives = 275.0, false_positives = 100.0, global_step = 19286, loss = 0.5854436, precision = 0.55947137, recall = 0.3159204, true_negatives = 5927.0, true_positives = 127.0


I0509 16:27:26.047033 140481974605568 tf_logging.py:115] Saving dict for global step 19286: auc = 0.64966416, eval_accuracy = 0.94167054, f1_score = 0.40381554, false_negatives = 275.0, false_positives = 100.0, global_step = 19286, loss = 0.5854436, precision = 0.55947137, recall = 0.3159204, true_negatives = 5927.0, true_positives = 127.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 19286: bert_category_confusion/model.ckpt-19286


I0509 16:27:26.049042 140481974605568 tf_logging.py:115] Saving 'checkpoint_path' summary for global step 19286: bert_category_confusion/model.ckpt-19286


{'auc': 0.64966416,
 'eval_accuracy': 0.94167054,
 'f1_score': 0.40381554,
 'false_negatives': 275.0,
 'false_positives': 100.0,
 'global_step': 19286,
 'loss': 0.5854436,
 'precision': 0.55947137,
 'recall': 0.3159204,
 'true_negatives': 5927.0,
 'true_positives': 127.0}

# Extracting the trained model


Now let's save our model:

In [80]:
def getPrediction(in_sentences):
  labels = ["Not an error", "Is an error"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x[0], text_b = x[1], label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  # predictions = estimator.predict(predict_input_fn)
  predictions = estimator.predict(predict_input_fn, checkpoint_path="./bert_category_confusion/model.ckpt-18320")
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [81]:
import nltk
import re
import itertools

nltk.download('perluniprops')
nltk.download('punkt')

[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/ivantorubarov/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/ivantorubarov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [82]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

from nltk.tokenize.moses import MosesDetokenizer
detokenizer = MosesDetokenizer()

def annotate(text):
  current_time = datetime.now()
  raw = text
  raw = re.sub(r'\s', ' ', raw)
  raw = re.sub(r'( )+', ' ', raw)
  sentences = nltk.sent_tokenize(raw)
  wordsoup = []
  annotation_set = []
  checking_ids = []
  tokens = []
  J = 0
  for sentence in sentences:
    wordsoup.append(tknzr.tokenize(sentence))
  for i in range(len(wordsoup)):
    tokenized_sentence = wordsoup[i]
    for j in range(len(tokenized_sentence)):
      token = tokenized_sentence[j]
      if not re.search(r'[a-zA-Z]', token):
        tokens.append([token, 0])
      else:
        tokens.append([token, None])
        substring = token
        si = i-2
        if si < 0:
          si = 0
        detokenizing_string = wordsoup[si:i] + [tokenized_sentence[:j] + ['[MASK]'] + tokenized_sentence[j+1:]] + wordsoup[i+1:i+3]
        detokenizing_string = [item for sublist in detokenizing_string for item in sublist]
        entry = detokenizer.detokenize(detokenizing_string, return_str=True)
        annotation_set.append([entry, substring])
        checking_ids.append(J)
      J += 1
  predictions = getPrediction(annotation_set[:128])
  for p in range(len(predictions)):
    if predictions[p][-1] == 'Is an error':
      tokens[checking_ids[p]][1] = 1
    else:
      tokens[checking_ids[p]][1] = 0
  print('\n')
  print("Annotation took time ", datetime.now() - current_time)
  return tokens

def print_annotated_webpage(tokens, title):
  out = '<html>\n<head>\n<title>'+title+'</title>\n<meta charset="utf-8">\n<style type="text/css">\n.blue {\n\tbackground: #a8d1ff;\n\tdisplay: inline-block;\n}\n</style>\n</head>\n<body>'
  outsoup = []
  for token in tokens:
    if token[1] == 1:
      token[0] = '<div class="blue">'+token[0]+'</div>'
    outsoup.append(token[0])
  bodystring = detokenizer.detokenize(outsoup, return_str=True)
  out += bodystring
  out += '</body>\n</html>'
  return out

In [89]:
import json

curname = "wsites_" + modelname + ".json"

test_texts = {"./data/exam/exam2014/EEm_7_1.txt": "The graph below illustrates how many people\u2019s procent of age 65 and over live and will be live between 1940 and 2040 in Japan, Sweden and USA.\n\nAs we can say, in all countries the proportion of aged 65 and over will be on the high level by 2040.\n\nIn detail look, we can see, that in Japan between 1940 and 1960 the proportion of old people was decreased from 5 procent to 3 procent. However, in Sweden it rose to 8 procent from 4 procent. Meanwhile, in USA there was 9 procent of people aged 65 and over in 1940, then in 1960 it increased by 10 procent. In Japan between 1960 and 1980 the proportion was at the same level. Unless, in Sweden and USA it rose to 14 and 15 percent. The graph below presents to us, that between 1983 and 2030 in Japan it rise from 3 procent to 10 procent, but in Sweden it is a little fall to 13 13 procent, but there was a high growth to 20 procent in 2010. By 2020 it will drop down to 18. In the USA, in 2020 the proportion will be steady. In 2040 the procent of old people will be on the high level: in Japan - 26 procent, in USA - 23, in Sweden 25.\n\nIn conclusion, we can see that the old population is rise up nowdays.\n", "./data/old IELTS/IELTS2016/EKu_145_1.txt": "The graph shows an amount of investement in renewable energy in the developing and developed countries. Also it represents the world total amount of spending money on the renewable energy. Starting from 2006 till 2008 an amount of investements in renewable energy was steadely rising in both the developed and developing countries. In 2008 it reached the index of about 110 Bn in the developed countries and about 60 Bn in the developing countries. An amount of money spending on renewable energy in 2009 in the developed countries decreased to the index of about 100 Bn, whereas the investements in the developing countries countinued to grow. In 2011 an amount of investements in the developed countries reached its peak of about 180 Bn. The maximum index of investements in renewable energy in the developing countries was in the next year in 2012, when the index was about 105 Bn. Also in 2012 the index of investements in developed countries began to decline and in 2013 it was about 125 Bn. Almost the same tendency was in the developing countries, where an amount of investements began to fall after the 2012 and in 2013 it was about 95 Bn. To sum up, the worlwide tendency of investement in renewable energy was that it reached its peak in the period between 2011 and 2012. After this period it bagan rapidly fall. Investement in renewable energy in the developing countries was strongly less than investement in the developed countries during the whole period between 2006 and 2013, though the tendency of increasing and falling was almoust the same in both developed and developing countries.", "./data/exam/exam2016/best_works/ZEv_2_2.txt": "It is known that air travel is one of the causes of air pollution and even global warming. Some people argue that part of flights for business and leisure is unnecessary, so governments should think about new laws to decrease the number of them. However, I suppose that this will harm society.\n\nAll of us are sometimes using planes as the mean of transport because it is easy, quick and comfortable way of travelling even on long distances. Air travel made our lifes easier as we don\u2019t need to spend months or even year to get from one place to another as it was centuries ago. Also we have opportunities to visit places that were untouched before. The costs of plane ticket are rather small so almost everybody can afford it. So if governments decide to cut down the number of flights, the cost of air travel will rise up rapidly and this will cause unability of some group of people to afford it. This may be harmful for their business if they have to travel often or if their family lives in the other part of the world. Moreover, it may affect pilots as a lot of them will be fired if the number of flights and air companies don\u2019t need so much work force.\n\nOn the other hand, the reduce of amount of the air travel may help to solve problems with air pollution and prevent future global warming as less planes will flight. However, I believe that this wouldn't make dramatic change in the problem of pollution as there are a lot of other things that harm our planet.\n\nIn conclusion I'd like to say that cutting down the amount of flights by introducing new laws may affect the air pollution but not greatly, while life of people may become uncomfortable and a lot of them will loose their job.\n", "./data/exam/exam2017_2/ABl_26_2.txt": "The problem of high popularity of pirate content appeared not so long ago but everyone has to deal with it in the postmodern era. There is still no universal answer for this question.\nI do believe that illegal connection to the pirate content should be punished. Firstly, copying and sharing pirate copies undermine the right of intellectual ownership. It leads to money loss and a violation of basic right to have a property. The situation goes even worse if we will not forget the fact that such types of content (music and films) are produced directly by the authors. Secondly, due to the high popularity of illegal content then is a reduction of investitions in the music and films productions. Every project needs to be financed properly and there is no exception. It is highly unlikely to create something special if there will not a necessary material base Tonit. Thirdly, consumers should pay for music and films because there is a strong point in the understanding of a connection between a spending of your own money and consuming goods. There is no debate in the society whether people should pay for air tickets or not. Music and films are not usually associated with classic private goods and it has to be changed.\nTo summarize, there some somewhat strong arguments to the punishment of law violators. These actions are unjust, selfish and ineffective. They may lead to dramatic reduction of content\u2019s quality. That would be a huge pity and a loss-loss situation.\n", "./data/exam/exam2017/OBy_149_2.txt": "Some part of humanity thinks that countries should import to other countries a minimum part of food which produces in this country, And the food should be the same value with food which population needs. I am disagree with this statement and I will try to explain why. \n\nFirst of all, there are some countries in the world which can not produce as much food as it is population needs. SO other countries can help such countries and it\u2019s people to survive by import some food to them. Also some countries have not got a special aquipment for producing any type of doof or in this region some species of plants cannot grow or kind of anymals cannot live. But this countryes can produce other type of resources. So countries can excnenge thir goods, or sell some products, with they have to earn money and buy food or something else from another country.  Secondly, if country can produce more food than it\u2019s population can eat, souch country will be able to sell a part of food which they don\u2019t need to earn some money and improve the live of population of this country or it can solve some problems which this county has (for example problems with bad roads). \n\nIn conclusion, I can say that it is really necessary for all of us and for all countries to help each other with food. That is why countries, which can produce more food than they need, should do that and give or sell this food to other countries. \n\n", "./data/exam/exam2017_5_2/EGe_226_2.txt": "There are many political and social programms to reduce crime and people have differents points of view about steps to be taken for reducing crime. Many of them suppose that the increase of prison sentences could help. However, others do not agree with them and think that there are better ways to solve the problem.\nThe arguments in protection of the position \"longer prison sentences\" are clear. Firstly, the long hard sentence as a punishment would make a potential criminal think twice before commiting a crime. However, even if the crime and the punishment would be uncomparable this step wouldn't have any result with dangerous people as maniacs etc. Secondly, The good argument is that as long the criminal keep being in prison as long the citizens are protected from him. This will make people feel safe. But at the same time the long sentence", "./data/exam/exam2017_5_2/EGe_146_1.txt": "The graphs below show the population size\u2019s changes in the USA. We can also see birth rate and death rate. According to the graphs, these processes are closely connected.\nFrom the very beginning, we can see that from 1750 to 1800 the number of people in the USA was quite stable. Death and birth rates were quite similar during this period of time and also remained stable. What is more, birth rate reached its peak in 1800. Then we can see the steady growing of population from 1800 to 1950. During this time death rate fell down dramatically from 1800 to 1850 and then continued to decrease. Birth rate during this period was also getting down until it reached the lowest point in 1950. Furthermore, from 1950 we see that the number of population jumped up while death rate remained the same from 1950 to 2000 and birth rate began to grow from 1950.\nTo conclude, we can say that from 1750 to 2000 the population of the USA has been growing up. During these years, there was a tendency in both birth and death rates to fall down. However, birth rate increased in the end of the period.", "./data/old IELTS/IELTS2016/EKu_61_1.txt": "The graph below demonstrate the changes in Ibvestment in Renewable Energy between thw ages 2006 and 2013. As an owerall trend investment increases from 100 d.Bn to 214. According to the graph developed contries always invest much more money than Developing contries. The overall increasing is not constant. It is slow between the marks 100 and 171, then we can notice a tiny decreasing to the 168. Then the line rapidly boomed up to the level of 279 d.Bn. After that we can see a constant decreasing from the 279 d.Bn to 214 d.Bn. As we have noticed above developed contries are more active in investment renewable energy. There is always a difference between developed and developing contries. It minimizes in the 2009 and 2013 years and consists about 40 d.Bn. Although there is a growth in both types of contries, the difference stays the same. The graph illustrate that in 2011 there is the biggest amount of investment in developed countries. It leveled out about 160 d.Bn, while the developing countries stayed at the 100d.Bn level. In summary we have to say that developed contries have the tendency to invest more money in Renewable Energy in the compare with Developing contries. As an overall trend both types of contryes increases the amount. But during the last years this trend is not underlying anymore.", "./data/exam/exam2014/AAl_24_2.txt": "Obviously, in our everyday life we often see the situation, when the number of male and female students is not at the same level in many universities. Of course, there are several reason, why boys usually choose other kind of proffession, than girls. But I suppose, there is no need to give a university instructions of what the propotion of male and female student should be.\nSome people consider, that having the equal number of boys and girls in the particular university will bring more possibilities for communication. Certainly, I can't disagree with this statement. Moreover, it will be better for student's social development. But we can't ignore the fact, that the main aim of high education is to bring up great proffesionals. So, I'll try to emphasize the most significant characteristics of why the equal ammount of males and females is not important for studying.\nFirst of all, everybody knows , that boys and girls tend to choose different sorts of proffessions. We can note it even at school, when usually boys are better in physics , mathematics , chemistry. While girls are likely to prefer literature , foreign lenguages, history. And this situation isn't bad, because it is natural.\nSecondly, in some regions this propotion is at high level because of some geographical reasons. For example, in China the population of men is great enough . As we can see, boys are born more frequently than girls.\nFurthemore, sometimes women don't want to get a high education level. So, they usually become housewifes, and they like to take care of their families and consider it as a profession.\nIn conclusion I would like to mention , that, according to my oppinion, the idea of acception equal number of boys and girls in universities seems to me unreliable.\n", "./data/exam/exam2016/ZEv_52_2.txt": "It is argued that ecological problems are one of the most vital, which include global warming and air pollution. There are a lot of reasons that can be grouped into one term \u2013 human activity. Atmosphere has been polluted by air travel for the last decade. There are some policies that should regulate and reduce the number of air travel. In my point of view, there is no right to limit people to move all over the world. In this essay will be discussed balanced between ecological problems and human right.\n\nNowadays we can observe globalization. A lot of people has the Internt to exchange their cultures, views and information. Thus migration has quickly scared for the last years. It became essential part of modern people\u2019s life because of technological progress. I think that government should not regulate human movement because it is not interruption of rights and private life. If government choose groups of people who can travel there would be discrimination for the rest people. In addition, a travel reduce related with business and economic declining.\n\nIn my opinion, there are a lot of ways to solve ecological problem. The prime example is to invest to develop alternative ecological sourse. Besides, government can introduce laws to air companies\u2019 standards not to reduce the amount of air travel. In conclusion, I would like to say that the air travel reduce is radical method to solve air pollution problem. As is observed there are more civil alternative ways. \n", "./data/exam/exam2014/AAl_31_2.txt": "It can be suggested that it is necessary to accept equal amount girls and boys in every subject in the Universities. However, this statement has some advantages and disadvantages. Nowadays, there is no doubt that disadvantages of this statement are less valuable than proccess.\nTo begin with the point about discrimination male and female students in Universitet , if they should study every subject in equal numbers. Probably, some of them do not want to study some subject , for example, girls do not prefer physical culture , at the same time , boys are keen on sport more than literature. As a result, there are some problems. One more of the main argument is that studing together male and female students can help to socialized them. Besides, it can be seen from the life, that men are good at one aspects and women are the best in other questions. They can help to each other to know something significant and usufull, because nobody knows all in this life. However, if was equal amount of male and female students, it can easy to work in pare. Unfortunatelly, not all students are prefer to work such way. Usually it is more pleasant for girls to communicate with the same category, because they have the same hobbies. More than that, usually male and female can embrassed each other and it also can a problem, if there are not the same numbers of men and women.\nTo sum up, it can be concluded, that the discrimination to all students in groups to the equal amount can break their writs. However, there are more advantages of this and it is possible to create some programms in Universities.\n", "./data/exam/exam2017_4/DPe_83_1.txt": "The bar chart shows the changes of unemployment rate in some world regions in 2014 and 2015. It has six different regions starting from N. Africa to Worldwide.\nAs we can see the maximum of unemployment people is in N. Africa. The rate reached the level of 12,5 per cent. The lawest percentage of unemployment people has S. Asia region. The average data accounts for EU.\nIt can be clearly seen, that there are difference beetween the 2014 and 2015. Countries as S. Asia, N. Africa and Worldwide have stable results of unemployment trends. In contrast, the situation in Middle East, EU and Latin America is different and percentage rate is flactuating. The most sharply drop of unemployment people happend in Middle East and Latin America regions. The decline is a mere 1-2%.\nThe worst worst and the best results of unemployment rate can be explained by the high level of poverty in N. Africa and the most modern technologies and lifestyle in S. Asia.\n", "./data/old IELTS/IELTS2016/EKu_132_1.txt": "On the graphs we can see changes of using Facebook from March 2012 to March 2013 and the main reasons for using it among men and women. The first diagram shows us that in March 2012 much more people used desktops to see their social pages \u2013 more than 140 million users and nearly 130 million people used phones. We can see that in March 2013 people did not like Facebook like before. Per cent of users was fall down. In March 2013 there are less than 60 million people used desktops and less than 100 users used mobiles. On the second diagram we see the main reasons for using Facebook. Women were more active users of Facebook, than men. Women like sharing photos or videos, seeing funny posts and learning ways to help others, men see that too, but much less. Only receiving updates men and women like both. In the end we can say, that users of Facebook are less with years and women are activity users than men in social nets.\u00a0", "./data/old IELTS/IELTS2015/EEm_38_1.txt": "The grath we are given illustrates the proportion between low and over 65-aged during the period since 1940-s till 2040-s.\n\nAs we can see, the grath shows examples of three countries: Japas, Sweden and USA. The firs thing we can point out is that number of people who is 65 and over has a visiable increase in the period around 2030-s  - 2040-s for every of three countries. The trends of grath\u2019s development for Sweden and USA are nearly the same. The key difference is that USA\u2019s trand is having a stable period during 2000-s - 2020-s while Sweden has rapid rise and then slight fall in population agen 65. This situation have caused difference in latest persents at 2040s. Sweden had 5% less 65 aged than USA and in 2050-s it will have 5% more.\n\nThe situation is different for Japan\u2019s trend. From 5 per sent of 65 aged their number falls till 2,5 per sent. Then, after stability on the level of 2,5 per sent in 1960-s - 1980-s the trend has a slight rise till 10% in 2030. And then it increases rapidly during 5 years till 25 and more per sent, which is more then Sweden\u2019s and USA\u2019s numbers.\n\nTo conclude, the overall trend is increasing proportion of 65-afed and over population in a few decades after 2020.\n", "./data/exam/exam2014/MBi_22_1.txt": "The graph shows the percent of people in the age of 65 and over in the period from 1940 till 2040 in Japan, Sweden and in the USA. It can be noticed that all countries have an increasing number of people aged 65 and over in the period from 2020 till 2040 \u2013 it has above 20% and above 25% in Japan.\nMorover, it can be said that Japan has less tham 5% till 2000. At that time Seden and the USA havr the similar population, which is from 7% till 17%. The beginning of 2000s is the most increasing period of time. The number of people aged 65 and over in Japan starts to increase from 5%, in the USA from 13%, in Sweden from 17%. It is daimed that the USA and Japan continue their increasing in the period from 2025 and than. However, the number of old people in Sweden decreases in the period from 2010 till 2025.\n"}
outie = [print_annotated_webpage(annotate(test_texts[tt]), tt) for tt in test_texts]
with open(curname, 'w', encoding="utf-8") as w:
  json.dump(outie, w)

INFO:tensorflow:Writing example 0 of 128


I0509 16:47:00.409162 140481974605568 tf_logging.py:115] Writing example 0 of 128


INFO:tensorflow:*** Example ***


I0509 16:47:00.411906 140481974605568 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0509 16:48:48.550853 140481974605568 tf_logging.py:115] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0509 16:48:48.624663 140481974605568 tf_logging.py:115] Done running local_init_op.




Annotation took time  0:00:07.238043


In [87]:
os.listdir(".")

['.ipython',
 '.cache',
 'bert_noun_number',
 '.ipynb_checkpoints',
 'bert_prepositions',
 'delete.json',
 '.jupyter',
 'Category_confusion_test.json',
 '.bashrc-anaconda3.bak',
 '.config',
 'REALEC_AutoAnnotator_BERT_pipeline Often_confused.ipynb',
 'Prepositions_test.json',
 'Noun_number_test.json',
 'wsites_OftenConfused_ToFifteen.json',
 'Often_confused_test.json',
 'Often_confused.json',
 'anaconda3',
 'Anaconda3-4.0.0-Linux-x86_64.sh',
 'Agreement_errors_test.json',
 'download_google_drive',
 'REALEC_AutoAnnotator_BERT_pipeline Noun number.ipynb',
 '.nano',
 '.bash_history',
 'wsites_Prepositions_ToFifteen.json',
 'Capitalisation.json',
 'Prepositions.json',
 'wsites_Noun_number_ToFifteen.json',
 'gdrive-linux-x64',
 '.keras',
 'delete_test.json',
 'Category_confusion.json',
 '.python_history',
 'nltk_data',
 'wsites_delete_ToFifteen.json',
 'wsites_Capitalisation_ToFifteen.json',
 'REALEC_AutoAnnotator_BERT_pipeline Confusion of categories.ipynb',
 'Datasets',
 '.local',
 'wsite

In [90]:
outie[5]

'<html>\n<head>\n<title>./data/exam/exam2014/AAl_24_2.txt</title>\n<meta charset="utf-8">\n<style type="text/css">\n.blue {\n\tbackground: #a8d1ff;\n\tdisplay: inline-block;\n}\n</style>\n</head>\n<body>Obviously, in our everyday life we often see the situation, when the number of male and female students is not at the same level in many universities. Of course, there are several reason, why boys usually choose other kind of proffession, than girls. But I suppose, there is no need to give a university instructions of what the propotion of male and female student should be. Some people consider, that having the equal number of boys and girls in the particular university will bring more possibilities for <div class="blue">communication</div>. Certainly, I can\'t disagree with this statement. Moreover, it will be better for student\'s social development. But we can\'t ignore the fact, that the main aim of high education is to bring up great proffesionals. So, I\'ll try to emphasize the mo

In [91]:
!./gdrive-linux-x64 upload wsites_Category_confusion_ToFifteen.json

Uploading wsites_Category_confusion_ToFifteen.json
Uploaded 1VbF_l2Elt6nptd2Ukn2QGUcztW0KTNtM at 21.2 KB/s, total 23.7 KB


In [74]:
!ls -lAh ./bert_category_confusion

total 6.5G
-rw-rw-r-- 1 ivantorubarov ivantorubarov  277 May  9 14:59 checkpoint
drwxr-xr-x 2 ivantorubarov ivantorubarov 4.0K May  9 13:12 eval
-rw-rw-r-- 1 ivantorubarov ivantorubarov 160M May  9 13:27 graph.pbtxt
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G May  9 14:46 model.ckpt-17320.data-00000-of-00001
-rw-rw-r-- 1 ivantorubarov ivantorubarov  23K May  9 14:46 model.ckpt-17320.index
-rw-rw-r-- 1 ivantorubarov ivantorubarov  44M May  9 14:46 model.ckpt-17320.meta
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G May  9 14:49 model.ckpt-17820.data-00000-of-00001
-rw-rw-r-- 1 ivantorubarov ivantorubarov  23K May  9 14:49 model.ckpt-17820.index
-rw-rw-r-- 1 ivantorubarov ivantorubarov  44M May  9 14:49 model.ckpt-17820.meta
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G May  9 14:52 model.ckpt-18320.data-00000-of-00001
-rw-rw-r-- 1 ivantorubarov ivantorubarov  23K May  9 14:52 model.ckpt-18320.index
-rw-rw-r-- 1 ivantorubarov ivantorubarov  44M May  9 14:53 model.ckpt-18320.meta
-rw-rw-

In [ ]:
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-2396.data-00000-of-00001
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-2396.index
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-2396.meta

Uploading ./bert_output/model.ckpt-2396.data-00000-of-00001
234.9 MB/1.3 GB, Rate: 41.1 MB/s                  

In [1]:
!rm -rf ./bert_output

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': curname})
uploaded.SetContentFile('./'+curname)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 16_pmUKOHS5lKXM2o-1QyVcPQohalCRZd


# Preparing existing model for REALEC production